# Atividade Prática — Aula 09: Docker + MLOps

**Objetivo:** Aprender Docker aplicado a MLOps através de exercícios práticos com dataset real.

**Entrega:** Notebook preenchido + evidências de execução

---

## PREPARAÇÃO INICIAL

### Passo 1: Criar estrutura do projeto
```bash
# Windows (PowerShell/CMD):
mkdir atividade-docker-mlops
cd atividade-docker-mlops
mkdir data, models, outputs, mlflow_data

# Windows (Git Bash) ou Linux:
mkdir atividade-docker-mlops
cd atividade-docker-mlops
mkdir -p data models outputs mlflow_data
```

### Passo 2: Copiar templates
Copie os arquivos da pasta `Aula 09/Templates/` para seu projeto.

### Passo 3: Verificar Docker
```bash
docker --version
docker-compose --version
```

**Pergunta:** Qual versão do Docker está instalada?
**Resposta:** [preencher]

---

## EXERCÍCIO 1: Jupyter em Container

### Passo 1: Subir Jupyter
```bash
# Windows (PowerShell):
docker run -d -p 8888:8888 -v ${PWD}:/workspace --name jupyter-mlops jupyter/scipy-notebook

# Windows (CMD):
docker run -d -p 8888:8888 -v "%cd%":/workspace --name jupyter-mlops jupyter/scipy-notebook

# Windows (Git Bash) ou Linux:
docker run -d -p 8888:8888 -v "$(pwd)":/workspace --name jupyter-mlops jupyter/scipy-notebook
```

### Passo 2: Obter token de acesso
```bash
docker logs jupyter-mlops
```
Copie o token que aparece no log.

### Passo 3: Acessar Jupyter
- Abra o browser em: http://localhost:8888
- Cole o token quando solicitado
- Crie um novo notebook Python

### Passo 4: Testar com dados
No notebook, execute:
```python
import pandas as pd
import numpy as np

# Criar dataset de exemplo
data = {
    'feature1': np.random.randn(100),
    'feature2': np.random.randn(100),
    'target': np.random.randint(0, 2, 100)
}
df = pd.DataFrame(data)
df.to_csv('/workspace/data/sample_data.csv', index=False)
print("Dataset criado com sucesso!")
print(f"Shape: {df.shape}")
print(f"Colunas: {df.columns.tolist()}")
```

### Passo 5: Verificar persistência
```bash
# Windows (PowerShell/CMD):
dir data
type data\sample_data.csv | more

# Windows (Git Bash) ou Linux:
ls -la data/
cat data/sample_data.csv | head -5
```

**Pergunta:** O arquivo foi salvo no host? Quantas linhas tem o dataset?
**Resposta:** [preencher]

**Screenshot obrigatório:** Jupyter rodando no browser
```
[ ] Capturado
```

---

## EXERCÍCIO 2: Treino em Batch

### Passo 1: Criar script de treino
Crie o arquivo `train.py`:
```python
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib
import json
import os

# Carregar dados
df = pd.read_csv('/workspace/data/sample_data.csv')
X = df[['feature1', 'feature2']]
y = df['target']

# Dividir dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Fazer predições
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Salvar modelo
os.makedirs('/workspace/models', exist_ok=True)
joblib.dump(model, '/workspace/models/model.pkl')

# Salvar métricas
metrics = {
    'accuracy': float(accuracy),
    'n_samples': len(df),
    'n_features': X.shape[1]
}
os.makedirs('/workspace/outputs', exist_ok=True)
with open('/workspace/outputs/metrics.json', 'w') as f:
    json.dump(metrics, f)

print(f"Modelo treinado com acurácia: {accuracy:.3f}")
print(f"Modelo salvo em: /workspace/models/model.pkl")
print(f"Métricas salvas em: /workspace/outputs/metrics.json")
```

### Passo 2: Executar treino em container
```bash
# Windows (PowerShell):
docker run -v ${PWD}:/workspace mlops-train:latest python train.py

# Windows (CMD):
docker run -v "%cd%":/workspace mlops-train:latest python train.py

# Windows (Git Bash) ou Linux:
docker run -v "$(pwd)":/workspace mlops-train:latest python train.py
```

### Passo 3: Verificar outputs
```bash
# Windows (PowerShell/CMD):
dir models
dir outputs
type outputs\metrics.json

# Windows (Git Bash) ou Linux:
ls -la models/
ls -la outputs/
cat outputs/metrics.json
```

**Pergunta:** Qual a acurácia do modelo? Quantas amostras foram usadas?
**Resposta:** [preencher]

**Screenshot obrigatório:** Treino executado com sucesso
```
[ ] Capturado
```

---

## EXERCÍCIO 3: Inferência em Batch

### Passo 1: Criar script de inferência
Crie o arquivo `predict.py`:
```python
import pandas as pd
import numpy as np
import joblib
import json
import os

# Carregar modelo
model = joblib.load('/workspace/models/model.pkl')

# Criar dados de teste
test_data = {
    'feature1': np.random.randn(20),
    'feature2': np.random.randn(20)
}
test_df = pd.DataFrame(test_data)

# Fazer predições
predictions = model.predict(test_df[['feature1', 'feature2']])
probabilities = model.predict_proba(test_df[['feature1', 'feature2']])

# Salvar resultados
results = pd.DataFrame({
    'feature1': test_df['feature1'],
    'feature2': test_df['feature2'],
    'prediction': predictions,
    'probability_class_0': probabilities[:, 0],
    'probability_class_1': probabilities[:, 1]
})

os.makedirs('/workspace/outputs', exist_ok=True)
results.to_csv('/workspace/outputs/predictions.csv', index=False)

print(f"Predições geradas: {len(predictions)} amostras")
print(f"Arquivo salvo em: /workspace/outputs/predictions.csv")
print(f"Primeiras 3 predições: {predictions[:3]}")
```

### Passo 2: Executar inferência em container
```bash
# Windows (PowerShell):
docker run -v ${PWD}:/workspace mlops-predict:latest python predict.py

# Windows (CMD):
docker run -v "%cd%":/workspace mlops-predict:latest python predict.py

# Windows (Git Bash) ou Linux:
docker run -v "$(pwd)":/workspace mlops-predict:latest python predict.py
```

### Passo 3: Verificar outputs
```bash
# Windows (PowerShell/CMD):
dir outputs
type outputs\predictions.csv | more

# Windows (Git Bash) ou Linux:
ls -la outputs/
cat outputs/predictions.csv | head -5
```

**Pergunta:** Quantas predições foram geradas? Qual a primeira predição?
**Resposta:** [preencher]

**Screenshot obrigatório:** Inferência executada com sucesso
```
[ ] Capturado
```

---

## EXERCÍCIO 4: MLflow Tracking

### Passo 1: Subir MLflow server
```bash
# Windows (PowerShell):
docker run -d -p 5000:5000 -v ${PWD}/mlflow_data:/mlflow --name mlflow-server mlflow/mlflow:latest

# Windows (CMD):
docker run -d -p 5000:5000 -v "%cd%\mlflow_data":/mlflow --name mlflow-server mlflow/mlflow:latest

# Windows (Git Bash) ou Linux:
docker run -d -p 5000:5000 -v "$(pwd)"/mlflow_data:/mlflow --name mlflow-server mlflow/mlflow:latest
```

### Passo 2: Acessar UI do MLflow
- Abra o browser em: http://localhost:5000
- Navegue pela interface

### Passo 3: Logar experimento
Crie o arquivo `mlflow_experiment.py`:
```python
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Configurar MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("docker-mlops-experiment")

# Carregar dados
df = pd.read_csv('/workspace/data/sample_data.csv')
X = df[['feature1', 'feature2']]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logar experimento
with mlflow.start_run():
    # Parâmetros
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("test_size", 0.2)
    
    # Treinar modelo
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Métricas
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("n_samples", len(df))
    
    # Modelo
    mlflow.sklearn.log_model(model, "model")
    
    print(f"Experimento logado com acurácia: {accuracy:.3f}")
```

### Passo 4: Executar experimento
```bash
# Windows (PowerShell):
docker run -v ${PWD}:/workspace mlops-train:latest python mlflow_experiment.py

# Windows (CMD):
docker run -v "%cd%":/workspace mlops-train:latest python mlflow_experiment.py

# Windows (Git Bash) ou Linux:
docker run -v "$(pwd)":/workspace mlops-train:latest python mlflow_experiment.py
```

### Passo 5: Verificar no MLflow UI
- Acesse http://localhost:5000
- Navegue para o experimento "docker-mlops-experiment"
- Verifique as métricas e parâmetros logados

**Pergunta:** Qual a acurácia logada no MLflow? Quantos experimentos aparecem?
**Resposta:** [preencher]

**Screenshot obrigatório:** MLflow UI funcionando
```
[ ] Capturado
```

---

## EXERCÍCIO 5: Compartilhar Imagem Docker

### Passo 1: Criar imagem personalizada
Crie o arquivo `Dockerfile`:
```dockerfile
FROM python:3.9-slim

WORKDIR /workspace

COPY requirements.txt .
RUN pip install -r requirements.txt

COPY . .

CMD ["python", "train.py"]
```

### Passo 2: Build da imagem
```bash
docker build -t meu-modelo-mlops:latest .
```

### Passo 3: Testar imagem localmente
```bash
# Windows (PowerShell):
docker run -v ${PWD}:/workspace meu-modelo-mlops:latest

# Windows (CMD):
docker run -v "%cd%":/workspace meu-modelo-mlops:latest

# Windows (Git Bash) ou Linux:
docker run -v "$(pwd)":/workspace meu-modelo-mlops:latest
```

### Passo 4: Salvar imagem em arquivo
```bash
docker save meu-modelo-mlops:latest -o meu-modelo-mlops.tar
```

### Passo 5: Compartilhar com colega
- Envie o arquivo `meu-modelo-mlops.tar` para um colega
- O colega deve executar:
```bash
docker load -i meu-modelo-mlops.tar
# Windows (PowerShell):
docker run -v ${PWD}:/workspace meu-modelo-mlops:latest
# Windows (CMD):
docker run -v "%cd%":/workspace meu-modelo-mlops:latest
# Windows (Git Bash) ou Linux:
docker run -v "$(pwd)":/workspace meu-modelo-mlops:latest
```

**Pergunta:** A imagem funcionou na máquina do colega? Qual o tamanho do arquivo .tar?
**Resposta:** [preencher]

**Screenshot obrigatório:** Imagem funcionando na máquina do colega
```
[ ] Capturado
```

---

## CHECKLIST FINAL

- [ ] Exercício 1: Jupyter funcionando com dados
- [ ] Exercício 2: Treino executado com métricas
- [ ] Exercício 3: Inferência gerando predições
- [ ] Exercício 4: MLflow logando experimentos
- [ ] Exercício 5: Imagem compartilhada funcionando
- [ ] Todas as perguntas respondidas
- [ ] Notebook salvo e entregue

**Data de entrega:** [preencher]
**Nome:** [preencher]
**Turma:** [preencher]
